In [1]:
!pip install gspread oauth2client wordcloud matplotlib --quiet

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import re
from IPython.display import clear_output



In [2]:
import json

with open("credentials.json") as f:
    creds = json.load(f)
    print("Email service account:", creds["client_email"])

Email service account: wordcloud-service@wordcloud-455309.iam.gserviceaccount.com


In [ ]:
#
# https://console.cloud.google.com/apis/credentials → Service Account Key → JSON
# `credentials.json`

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# 🔗 Table
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Sw5-evFZCtmEUygScHljHp3y4B7Gxh-pw2Izu-cK0Kg/edit?resourcekey#gid=1732050705"
sheet = client.open_by_url(spreadsheet_url).sheet1

# 🔁 Update
while True:
    clear_output(wait=True)
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    if df.empty:
        print("Waiting responses...")
    else:
        text_data = df.iloc[:, 1:].astype(str).values.flatten()
        text = ' '.join(text_data).strip()
        words = re.findall(r'\b\w+\b', text)
        cleaned_words = [word for word in words if word.lower() not in STOPWORDS and word.isalpha()]


        if not cleaned_words:
          print(" Form is ready, but is no sutable words for cloud")

        else:
          cleaned_text = ' '.join(cleaned_words)
          wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(cleaned_text)

          plt.figure(figsize=(12, 6))
          plt.imshow(wordcloud, interpolation='bilinear')
          plt.axis("off")
          plt.title("Word cloud: AI")
          plt.show()

    time.sleep(10)

Waiting responses...
